In [8]:
from PIL import Image
import numpy as np
import sys
from configparser import ConfigParser
import argparse
import os
from os.path import isfile, join
import warnings

# class PhotoCommander:
#     def __init__(self, image_path):
#         self.image_path = image_path
#         self.image = Image.open(image_path)
#         #self.image = Image.new(io.mode, io.size)
        
#     def show(self):
#         return self.image.show()
        
#     def rotate_90_deg(self, rotate_count = 1):
#         width, height = self.image.size
#         if(rotate_count % 4 == 0):
#             return self
#         elif(rotate_count % 4 == 1):
#             self.image = self.image.transpose(Image.ROTATE_90).rotate(180)
#         elif(rotate_count % 4 == 2):
#             self.image = self.image.rotate(180)
#         else:
#             self.image = self.image.transpose(Image.ROTATE_90)
#         return self
    
#     def filter(self, amount, rgb='r'):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][0] if 'r' not in rgb else int(pixels[i,j][0] * amount),
#                     pixels[i,j][1] if 'g' not in rgb else int(pixels[i,j][1] * amount),
#                     pixels[i,j][2] if 'b' not in rgb else int(pixels[i,j][2] * amount)
#                 )
#         return self  
    
#     def set_color(self, value, rgb='r'):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][0] if 'r' not in rgb else value,
#                     pixels[i,j][1] if 'g' not in rgb else value,
#                     pixels[i,j][2] if 'b' not in rgb else value
#                 )
#         return self 
    
#     def rotate_color(self):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][1],
#                     pixels[i,j][2],
#                     pixels[i,j][0]
#                 )
#         return self 
    
#     def invert(self, rgb='r'):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][0] if 'r' not in rgb else 255-pixels[i,j][0],
#                     pixels[i,j][1] if 'g' not in rgb else 255-pixels[i,j][1],
#                     pixels[i,j][2] if 'b' not in rgb else 255-pixels[i,j][2]
#                 )
#         return self 
    
#     def grayscale_red(self):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][0],
#                     pixels[i,j][0],
#                     pixels[i,j][0]
#                 )
#         return self 
    
#     def grayscale_green(self):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][1],
#                     pixels[i,j][1],
#                     pixels[i,j][1]
#                 )
#         return self 
    
#     def grayscale_blue(self):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     pixels[i,j][2],
#                     pixels[i,j][2],
#                     pixels[i,j][2]
#                 )
#         return self 
    
#     def grayscale(self):
#         pixels = self.image.load()
        
#         for i in range(self.image.size[0]):
#             for j in range(self.image.size[1]):
#                 pixels[i,j] = (
#                     int((pixels[i,j][0] + pixels[i,j][1] + pixels[i,j][2])/3),
#                     int((pixels[i,j][0] + pixels[i,j][1] + pixels[i,j][2])/3),
#                     int((pixels[i,j][0] + pixels[i,j][1] + pixels[i,j][2])/3)
#                 )
#         return self 

def is_filetype(path, types=['.jpg', '.jpeg', '.png']):
    for type in types:
        if path.endswith(type):
            return True
    return False

In [9]:
# img = PhotoCommander(r"C:\Users\kolth\Downloads\IMG_20191108_090307.jpg")
# img.grayscale().show()

In [10]:
config = ConfigParser()
config.read('config.ini')

if not os.path.exists('config.ini'):
    config.add_section('settings')
    config.set('settings', 'destination', './')
    config.set('settings', 'history-enabled', 'False')

    with open('config.ini', 'w') as f:
        config.write(f)

destination = config.get('settings', 'destination')
print(destination)

./


In [42]:
args = "-d ./photos --verbose".split()

parser = argparse.ArgumentParser("Python Photo Commander")

# Adds command-line arguments
parser.add_argument('--default-destination', help='set the default image/folder for commands without destination specified')
parser.add_argument('-d', '--destination', help='sets the current command\'s destination')
parser.add_argument('-v', '--verbose', help='increases output verbosity', action='store_true')
args = parser.parse_args(args)

# Checks for destination commands and either sets the default or regular destination.
# A regular destination will take priority over a default in all cases
destination = config.get('settings', 'destination')
if args.default_destination is not None:
    destination = args.default_destination
    config.set('settings', 'destination', destination)
if args.destination != None:
    destination = args.destination
dest_type = "folder" if os.path.isdir(destination) else "file"
if not os.path.exists(destination):
    dest_type = "does not exist"

photos = []
if dest_type == 'folder':
    photos = [os.path.relpath(os.path.join('./', f)) for f in os.listdir(destination) 
              if isfile(join(destination, f)) and is_filetype(f)]
    if len(photos) <= 0:
        warnings.warn('Warning: the destination folder \'{}\' does not contain any images. (.jpg, .png)'.format(destination))
elif dest_type == 'file':
    if is_filetype(destination):
        photos = [os.path.relpath(os.path.join('./', destination))]
    else:
        raise ValueError('If passing in a file, it must be either .jpg or .png')
    
if args.verbose:
    print("Destination: {} ({})".format(destination, dest_type))
    print("History Enabled: False")
    print(photos)

Destination: ./photos (folder)
History Enabled: False
['beautiful-scenery.jpg', 'mountain-scenery.jpg']
